In [1]:
#import base64 # You need it if you define binary uploads
#from __future__ import print_function # py 2.7 compat.
import ipywidgets as widgets # Widget definitions.
from traitlets import List, Unicode  # Traitlets needed to add synced attributes to the widget.

class LoadFileWidget(widgets.DOMWidget):
    _view_name = Unicode('LoadFileView').tag(sync=True)
    _view_module = Unicode('loadfile').tag(sync=True)
    filenames = List([]).tag(sync=True)
    #base64_data = Unicode().tag(sync=True)
    fcontents = List([]).tag(sync=True)
    #fcontents = Unicode().tag(sync=True)

    def __init__(self, **kwargs):
        """Constructor"""
        super().__init__(**kwargs)

        # Allow the user to register error callbacks with the following signatures:
        #    callback()
        #    callback(sender)
        self.errors = widgets.CallbackDispatcher(accepted_nargs=[0, 1])

        # Listen for custom msgs
        self.on_msg(self._handle_custom_msg)

    def _handle_custom_msg(self, content):
        """Handle a msg from the front-end.

        Parameters
        ----------
        content: dict
            Content of the msg."""
        if 'event' in content and content['event'] == 'error':
            self.errors()
            self.errors(self)

In [57]:
%%javascript
requirejs.undef('loadfile');

define('loadfile', ["jupyter-js-widgets"], function(widgets) {

    var LoadFileView = widgets.DOMWidgetView.extend({
        render: function(){
            // Render the view using HTML5 multiple file input support.
            /*this.setElement($('<input class="fileinput" multiple="multiple" name="datafile"  />')
                .attr('type', 'file'));*/
            
            //this.$drop = $('<div style="border-color:green; border-width: 5px; border-style: solid;">
            //<label for="files" class="btn">Click or drop files</label>
            //<input id="files" class="fileinput" multiple="multiple" name="datafile" style="visibility:hidden;" type="file"></div>');
            //this.setElement($this.$drop);
            //this.$drop_start=$('');
            this.$select=$('<input class="fileinput" multiple="multiple" name="datafile" /*style="visibility:hidden;"*/ />').attr('type', 'file');
            this.$drop=$('<div style="border-color:green; border-width: 5px; border-style: solid;" id="drop">or drop files here.</div>');
            //this.$el.append(this.$drop_start);
            this.$el.append(this.$select);
            this.$el.append(this.$drop);
            
            this.$output=$('<div id="out"> </div>');
            this.$el.append(this.$output);
            
        },

        events: {
            // List of events and their handlers.
            'change': 'handle_file_change',
            'drop': 'handle_drop_files',
            'dragover': 'handle_drag_over',
        },

        handle_drop_files: function(evt) {
            evt.stopPropagation();
            evt.preventDefault();
 
            var files = evt.originalEvent.dataTransfer.files; // FileList Objekt
            this.handle_files(files);
        },
        
        handle_drag_over: function(evt) {
            evt.stopPropagation();
            evt.preventDefault();
            evt.originalEvent.dataTransfer.dropEffect = 'copy'; 
        },
        
        handle_file_change: function(evt) { 
            // Handle when the user has changed the file.
            // Retrieve the FileList object
            var files = evt.originalEvent.target.files;
            this.handle_files(files);
        },
        
        handle_files: function(files) {
            // Save context (or namespace or whatever this is)
            var that = this;

            
            var filenames = [];
            var file_readers = [];
            var fcont = [];
            
            
            console.log("Reading files:");

            for (var i = 0; i < files.length; i++) {
                var file = files[i];
                console.log("Filename: " + file.name);
                console.log("Type: " + file.type);
                console.log("Size: " + file.size + " bytes");
                filenames.push(file.name);
                

                // Read the file's textual content and set value_i to those contents.
                file_readers.push(new FileReader());
                file_readers[i].onload = (function(file, i) {
                    return function(e) {
                        that.model.set('value_' + i, e.target.result);
                        //that.touch();
                        fcont.push(e.target.result);
                        console.log(fcont);
                        // last file?
                        if(i==files.length-1) {
                            that.model.set('fcontents', fcont);
                        }    
                        that.touch();           
                        console.log("file_" + i + " loaded: " + file.name);
                    };
                })(file, i);

                file_readers[i].readAsText(file);
            }

            var output = [];
            for (var i = 0, f; f = files[i]; i++) {
                var fdate = new Date(f.lastModified);
                output.push('<li><strong>', escape(f.name), '</strong> (', f.type || 'n/a', ') - ',
                  f.size, ' bytes, last modified: ',
                  fdate.toISOString(), '</li>');
            }
            document.getElementById('out').innerHTML = '<ul>' + output.join('') + '</ul>';
            
            // Set the filenames of the files.
            this.model.set('filenames', filenames);
            this.touch();
        }    
    });

    // Register the LoadFileView with the widget manager.
    return {
        LoadFileView: LoadFileView
    };
});

<IPython.core.display.Javascript object>

In [58]:
file_widget = LoadFileWidget()

def file_loaded(change):
    '''Register an event to save contents when a file has been uploaded.'''
    #print(change['new'])
    i = int(change['name'].split('_')[1])
    fname = file_widget.filenames[i]
    #print('file_loaded: {}'.format(fname))

def file_loading(change):
    '''Update self.model when user requests a list of files to be uploaded'''
    #print(change['new'])
    num = len(change['new'])
    traits = [('value_{}'.format(i), Unicode().tag(sync=True)) for i in range(num)]
    file_widget.add_traits(**dict(traits))
    for i in range(num):
        file_widget.observe(file_loaded, 'value_{}'.format(i))

file_widget.observe(file_loading, names='filenames')

def file_failed():
    print("Could not load some file contents.")

file_widget.errors.register_callback(file_failed)


file_widget

In [60]:
file_widget.fcontents[0]

'Dr. Michael Wack\r\nDepartment of Earth and Environmental Sciences\r\nGeophysics\r\nLMU Munich\r\nTheresienstr. 41/IV\r\nD-80333 Munich\r\nGermany\r\nTel: +49 (0) 89 2180 4238\r\nFax: +49 (0) 89 2180 4205'

In [61]:
file_widget.filenames

['Signatur_Geophysik.txt', 'tonegen.c']